In [1]:
import spacy
import pandas as pd
import numpy as np
import nltk
from nltk.tokenize.toktok import ToktokTokenizer
import re
from bs4 import BeautifulSoup
from contractions import CONTRACTION_MAP
import unicodedata
import en_core_web_sm
import os
import matplotlib.pyplot as plt
import seaborn as sns
import statsmodels.api as sm
from wordcloud import WordCloud, STOPWORDS
%matplotlib inline
import numpy as np
import scipy as sp
import matplotlib as mpl
import matplotlib.cm as cm
from six.moves import range
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split
from sklearn import linear_model
from sklearn.linear_model import Ridge
from sklearn.linear_model import LinearRegression
from sklearn import svm
from time import process_time, time
from sklearn import metrics
from sklearn.svm import SVR
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import GradientBoostingRegressor

# Setup Pandas
pd.set_option('display.width', 500)
pd.set_option('display.max_columns', 100)
pd.set_option('display.notebook_repr_html', True)

# Setup Seaborn
sns.set_style("whitegrid")
sns.set_context("poster")

In [2]:
os.chdir('C:/Users/adamj/Documents/Github/Springboard/NLP Projects/CSV Files')

In [3]:
goodreads=open('Goodreads_preprocessed2.csv','r',encoding='utf8')
goodreads_csv=pd.read_csv(goodreads)

In [4]:
goodreads_csv.head()

,Unnamed: 0,title,author,author_url,rating,voters_count,url,into_movie,gender,hometown,books-num,born-at,works-count,publication-date,num-pages,popular-shelves,rating-dist,publisher,book-description,5 star votes,4 star votes,3 star votes,2 star votes,1 star votes,Total Votes,title_clean,book-series,book-num-in-series,% 5 star votes,% 4 star votes,% 3 star votes,% 2 star votes,% 1 star votes,title_pre-p,book-desc_pre-p,pop-shelves_pre-p,author_pre-p,gender_mf
0,0,"The Sword and The Prophet (Syren, #1)",Missy LaRae,https://www.goodreads.com/author/show/5805389....,3.80,50,https://goodreads.com/book/show/13575987-the-s...,0,female,"Portland, Oregon",2,12/17/1979,2,3/29/2012,150.0,young-adult fantasy library kindle series curr...,5:19|4:13|3:9|2:7|1:2|total:50,Pretty Wycked Designs,"“Why'd Mama hate us so much, Tyler?"" My head r...",19,13,9,7,2,50,The Sword and The Prophet,Syren,1,0.380000,0.260000,0.180000,0.140000,0.040000,sword prophet,whyd mama hate us much tyler head rested chest...,[young-adult fantasy library kindle series cur...,missy larae,female
1,1,"Ruins (Partials Sequence, #3)",Dan Wells,https://www.goodreads.com/author/show/2740668....,4.05,14673,https://goodreads.com/book/show/17607716-ruins,0,male,NaN,10,NaN,45,3/11/2014,464.0,young-adult dystopian currently-reading dystop...,5:5367|4:5621|3:2954|2:587|1:150|total:14679,Balzer + Bray,"Kira, Samm, and Marcus fight to prevent a fina...",5367,5621,2954,587,150,14679,Ruins,Partials Sequence,3,0.365624,0.382928,0.201240,0.039989,0.010219,ruins,kira samm marcus fight prevent final war parti...,[young-adult dystopian currently-reading dysto...,dan wells,male
2,2,"Breathe (Sea Breeze, #1)",Abbi Glines,https://www.goodreads.com/author/show/4872191....,3.86,77770,https://goodreads.com/book/show/11479285-breathe,0,female,"Birmingham, Alabama",10,NaN,88,5/16/2011,164.0,romance currently-reading young-adult favorite...,5:26825|4:24784|3:17639|2:5881|1:2655|total:77784,Wild Child Publishing,Sadie White's summer job isn't going to be on ...,26825,24784,17639,5881,2655,77784,Breathe,Sea Breeze,1,0.344865,0.318626,0.226769,0.075607,0.034133,breathe,sadie whites summer job not going beach life g...,[romance currently-reading young-adult favorit...,abbi glines,female
3,4,The Lock Artist,Steve Hamilton,https://www.goodreads.com/author/show/9891.Ste...,3.97,10967,https://goodreads.com/book/show/6811221-the-lo...,0,male,"Detroit, MI",10,NaN,35,NaN,NaN,currently-reading mystery fiction favorites cr...,5:3307|4:4786|3:2239|2:494|1:148|total:10974,NaN,"<i>""I was the Miracle Boy, once upon a time. L...",3307,4786,2239,494,148,10974,The Lock Artist,NaN,NaN,0.301349,0.436122,0.204028,0.045015,0.013486,lock artist,miracle boy upon time later milford mute golde...,[currently-reading mystery fiction favorites c...,steve hamilton,male
4,5,"Captivated (The Affliction Series, #1)",E.M. Jade,https://www.goodreads.com/author/show/4784370....,3.90,2818,https://goodreads.com/book/show/11082489-capti...,0,female,Phoenix,6,1/30/1986,6,12/1/2011,375.0,currently-reading kindle paranormal vampires y...,5:1072|4:834|3:590|2:203|1:119|total:2818,Eminent Books,“I stood close to the edge looking down at wha...,1072,834,590,203,119,2818,Captivated,The Affliction Series,1,0.380412,0.295955,0.209368,0.072037,0.042229,captivated,stood close edge looking thought death somehow...,[currently-reading kindle paranormal vampires ...,e jade,female


In [5]:
goodreads_csv.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 27846 entries, 0 to 27845
Data columns (total 38 columns):
Unnamed: 0            27846 non-null int64
title                 27846 non-null object
author                27846 non-null object
author_url            27846 non-null object
rating                27846 non-null float64
voters_count          27846 non-null int64
url                   27846 non-null object
into_movie            27846 non-null int64
gender                23049 non-null object
hometown              14698 non-null object
books-num             27846 non-null int64
born-at               7786 non-null object
works-count           27846 non-null int64
publication-date      22732 non-null object
num-pages             24149 non-null float64
popular-shelves       27846 non-null object
rating-dist           27846 non-null object
publisher             23499 non-null object
book-description      27846 non-null object
5 star votes          27846 non-null int64
4 star votes    

In [6]:
goodreads_ML=goodreads_csv[["rating","into_movie","num-pages","5 star votes","4 star votes","3 star votes","2 star votes","1 star votes","Total Votes","% 5 star votes","% 4 star votes","% 3 star votes","% 2 star votes","% 1 star votes","title_pre-p","book-desc_pre-p","author_pre-p","gender_mf"]]

In [7]:
goodreads_ML.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 27846 entries, 0 to 27845
Data columns (total 18 columns):
rating             27846 non-null float64
into_movie         27846 non-null int64
num-pages          24149 non-null float64
5 star votes       27846 non-null int64
4 star votes       27846 non-null int64
3 star votes       27846 non-null int64
2 star votes       27846 non-null int64
1 star votes       27846 non-null int64
Total Votes        27846 non-null int64
% 5 star votes     27834 non-null float64
% 4 star votes     27834 non-null float64
% 3 star votes     27834 non-null float64
% 2 star votes     27834 non-null float64
% 1 star votes     27834 non-null float64
title_pre-p        27617 non-null object
book-desc_pre-p    27801 non-null object
author_pre-p       27790 non-null object
gender_mf          23036 non-null object
dtypes: float64(7), int64(7), object(4)
memory usage: 3.8+ MB


In [8]:
goodreads_ML['book-desc_pre-p'].head(10)

0    whyd mama hate us much tyler head rested chest...
1    kira samm marcus fight prevent final war parti...
2    sadie whites summer job not going beach life g...
3    miracle boy upon time later milford mute golde...
4    stood close edge looking thought death somehow...
5    stunning first novel literary thriller retired...
6    autumn inconceivable comes blackeberg suburb s...
7    anches stones daggers bones  locked beast away...
8    gia montgomerys boyfriend adley dumps parking ...
9    novel ideas book year  top ten goodreads listo...
Name: book-desc_pre-p, dtype: object

In [9]:
goodreads_ML.isna().sum()

rating                0
into_movie            0
num-pages          3697
5 star votes          0
4 star votes          0
3 star votes          0
2 star votes          0
1 star votes          0
Total Votes           0
% 5 star votes       12
% 4 star votes       12
% 3 star votes       12
% 2 star votes       12
% 1 star votes       12
title_pre-p         229
book-desc_pre-p      45
author_pre-p         56
gender_mf          4810
dtype: int64

In [10]:
goodreads_ML_nona=goodreads_ML.dropna()


In [11]:
goodreads_ML_nona.isna().sum()

rating             0
into_movie         0
num-pages          0
5 star votes       0
4 star votes       0
3 star votes       0
2 star votes       0
1 star votes       0
Total Votes        0
% 5 star votes     0
% 4 star votes     0
% 3 star votes     0
% 2 star votes     0
% 1 star votes     0
title_pre-p        0
book-desc_pre-p    0
author_pre-p       0
gender_mf          0
dtype: int64

In [12]:
goodreads_ML_nona.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 20017 entries, 0 to 27843
Data columns (total 18 columns):
rating             20017 non-null float64
into_movie         20017 non-null int64
num-pages          20017 non-null float64
5 star votes       20017 non-null int64
4 star votes       20017 non-null int64
3 star votes       20017 non-null int64
2 star votes       20017 non-null int64
1 star votes       20017 non-null int64
Total Votes        20017 non-null int64
% 5 star votes     20017 non-null float64
% 4 star votes     20017 non-null float64
% 3 star votes     20017 non-null float64
% 2 star votes     20017 non-null float64
% 1 star votes     20017 non-null float64
title_pre-p        20017 non-null object
book-desc_pre-p    20017 non-null object
author_pre-p       20017 non-null object
gender_mf          20017 non-null object
dtypes: float64(7), int64(7), object(4)
memory usage: 2.9+ MB


In [13]:
print(goodreads_ML_nona['rating'].describe())

count    20017.000000
mean         3.990384
std          0.311906
min          1.000000
25%          3.800000
50%          3.990000
75%          4.180000
max          5.000000
Name: rating, dtype: float64


In [14]:
#create feature data frame and isolate target variable

#feature dataframe
X = goodreads_ML_nona['book-desc_pre-p'] # load the dataset as a pandas data frame

#target variable
y = goodreads_ML_nona['rating']

# create training and testing vars
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2,random_state=27)

In [15]:
X_train

1474     jules happy saved lot peoples lives shes final...
14265    div  href http bp blogspot com tj crstsige ucr...
11517    stepped cliff falling madness   mind bending p...
6705     one thing depend cold sassy georgia word gets ...
17628    b debut novel pulses humor empathy explores he...
                               ...                        
22243    scottish lowlands october   year come gone sin...
6857     b move mr darcy theres new bachelor town b   s...
20839    melkorka princess first daughter magnificent k...
5535     strong erin hunters nationally bestselling war...
7261     penelope dupres lives affluent suburb new york...
Name: book-desc_pre-p, Length: 16013, dtype: object

In [16]:
len(y_train)

16013

In [17]:
len(X_test)

4004

In [18]:
#convert text data to bag of words
vectorizer = CountVectorizer(min_df=1)

#step 1 - convert text data to bag of words (fit _transform() on train data , use transform() on test data)
X_train_BoW = vectorizer.fit_transform(X_train) #fit transform is for learning
X_test_BoW = vectorizer.transform(X_test) #not learning the vocabulary again

In [19]:
vectorizer.get_feature_names()

['__',
 '_____',
 '_______',
 '__________',
 '_______________',
 '__________________________',
 '___________________________',
 '_____________________________',
 '________________________________',
 '_________________________________',
 '__________________________________',
 '____________________________________',
 '____________________________________________',
 '____________________________________________________________________________________',
 '__director_s_cut',
 '_all_washed_up_and_nowhere_to_run',
 '_beverly',
 '_blank',
 '_cover',
 '_encoding',
 '_graustark_',
 '_here',
 '_orderelyon',
 '_project',
 '_rise',
 '_signs',
 '_the',
 '_v',
 'a_clash_of_kings',
 'a_shared_range',
 'aa',
 'aaaaaaaaa',
 'aabc',
 'aace',
 'aachim',
 'aaham',
 'aahams',
 'aahamss',
 'aakadon',
 'aakasear',
 'aam',
 'aamir',
 'aamm',
 'aams',
 'aan',
 'aanbod',
 'aangaan',
 'aangename',
 'aangericht',
 'aangewakkerde',
 'aanklacht',
 'aanstormende',
 'aantal',
 'aanvallen',
 'aanvang',
 'aarde',
 'aaro

In [20]:
X_train_BoW

<16013x87420 sparse matrix of type '<class 'numpy.int64'>'
	with 1211686 stored elements in Compressed Sparse Row format>

In [ ]:
#1 Linear Regression

#train the algorithm
regressor = LinearRegression()
regressor.fit(X_train_BoW, y_train)

In [ ]:
#use our test data and see how accurately our algorithm predicts the rating score
y_pred = regressor.predict(X_test_BoW)

In [ ]:
#Evaluating the Algorithm
print('Mean Absolute Error:', metrics.mean_absolute_error(y_test, y_pred))
print('Mean Squared Error:', metrics.mean_squared_error(y_test, y_pred))
print('Root Mean Squared Error:', np.sqrt(metrics.mean_squared_error(y_test, y_pred)))

You can see that the value of root mean squared error is 0.506, which is 12% of the mean value of the ratings of all the books, i.e. 3.99. This means that our algorithm did a substandard job.

In [ ]:
#2 SVM regression

#Feature Scaling
from sklearn.preprocessing import StandardScaler

sc_y = StandardScaler()
y = sc_y.fit_transform(y_train)

In [ ]:
from sklearn.svm import SVR
regressor = SVR(kernel = 'rbf',gamma='scale')
regressor.fit(X_train_BoW, y_train)

In [ ]:
y_pred = regressor.predict(X_test_BoW)

In [ ]:
#Evaluating the Algorithm
print('Mean Absolute Error:', metrics.mean_absolute_error(y_test, y_pred))
print('Mean Squared Error:', metrics.mean_squared_error(y_test, y_pred))
print('Root Mean Squared Error:', np.sqrt(metrics.mean_squared_error(y_test, y_pred)))

You can see that the value of root mean squared error is 0.304, which is 7.6% of the mean value of the ratings of all the books, i.e. 3.99. This means that our algorithm did a good job. Better than the linear regression, at least.

In [ ]:
#3 Fitting the Decision Tree Regression Model to the dataset
# Create the Decision Tree regressor object here
#DecisionTreeRegressor class has many parameters. Input only #random_state=0 or 42.
regressor = DecisionTreeRegressor(random_state=0)

In [ ]:
#Fit the regressor object to the dataset.
regressor.fit(X_train_BoW,y_train)

In [ ]:
#Predicting a new result
y_pred = regressor.predict(X_test_BoW)

In [ ]:
#Evaluating the Algorithm
print('Mean Absolute Error:', metrics.mean_absolute_error(y_test, y_pred))
print('Mean Squared Error:', metrics.mean_squared_error(y_test, y_pred))
print('Root Mean Squared Error:', np.sqrt(metrics.mean_squared_error(y_test, y_pred)))

You can see that the value of root mean squared error is 0.398, which is about 10% of the mean value of the ratings of all the books, i.e. 3.99. This means that our algorithm did a decent job. Better than the linear regression but worse than SVM.

In [ ]:
#4 Random Forest

# Import the model we are using
# Instantiate model with 1000 decision trees
rf = RandomForestRegressor(n_estimators = 100, random_state = 42)
# Train the model on training data
rf.fit(X_train_BoW, y_train);


In [ ]:
#Predicting a new result
y_pred = rf.predict(X_test_BoW)

In [ ]:
#Evaluating the Algorithm
print('Mean Absolute Error:', metrics.mean_absolute_error(y_test, y_pred))
print('Mean Squared Error:', metrics.mean_squared_error(y_test, y_pred))
print('Root Mean Squared Error:', np.sqrt(metrics.mean_squared_error(y_test, y_pred)))

You can see that the value of root mean squared error is 0.292, which is 7.3% of the mean value of the ratings of all the books, i.e. 3.99. This means that our algorithm did a good job. Better than the linear regression, decision tree, and SVM algorithms.

In [ ]:
#5 Gradient Boosting

#import the model we are using

#model
gradient_boosting_regressor_model= GradientBoostingRegressor()
#train the model
gradient_boosting_regressor_model.fit(X_train_BoW, y_train)

In [ ]:
#Predicting a new result
y_pred = gradient_boosting_regressor_model.predict(X_test_BoW)

In [ ]:
#Evaluating the Algorithm
print('Mean Absolute Error:', metrics.mean_absolute_error(y_test, y_pred))
print('Mean Squared Error:', metrics.mean_squared_error(y_test, y_pred))
print('Root Mean Squared Error:', np.sqrt(metrics.mean_squared_error(y_test, y_pred)))

You can see that the value of root mean squared error is 0.298, which is 7.5% of the mean value of the ratings of all the books, i.e. 3.99. This means that our algorithm did a good job. Better than the linear regression and decision tree algorithms. Essentially the same as Random Forest (slightly worse) and SVM (slightly better).

Here are the final rankings of the models with just the single feature on the target variable:
    1. Random Forest (rMSE: .292)
    2. Gradient Boosting (rMSE: .298)
    3. Support Vector Machine (rMSE: .305)
    4. Decision Trees (rMSE: .398)
    5. Linear Regression (rMSE: .482)

We will now use Grid Search CV to choose the best hyperparameters for our top two models: Random Forest and Gradient Boosting.

In [ ]:
#tune hyperparameters for Random Forest
#n_estimators = number of trees in the foreset
#max_features = max number of features considered for splitting a node
#max_depth = max number of levels in each decision tree
#min_samples_split = min number of data points placed in a node before the node is split
#min_samples_leaf = min number of data points allowed in a leaf node
#bootstrap = method for sampling data points (with or without replacement)

In [21]:
from sklearn.ensemble import RandomForestRegressor
rf = RandomForestRegressor(random_state = 42)
from pprint import pprint
# Look at parameters used by our current forest
print('Parameters currently in use:\n')
pprint(rf.get_params())

Parameters currently in use:

{'bootstrap': True,
 'criterion': 'mse',
 'max_depth': None,
 'max_features': 'auto',
 'max_leaf_nodes': None,
 'min_impurity_decrease': 0.0,
 'min_impurity_split': None,
 'min_samples_leaf': 1,
 'min_samples_split': 2,
 'min_weight_fraction_leaf': 0.0,
 'n_estimators': 'warn',
 'n_jobs': None,
 'oob_score': False,
 'random_state': 42,
 'verbose': 0,
 'warm_start': False}


In [ ]:
#from sklearn.model_selection import RandomizedSearchCV

#lets tune max_depth, n_estimators, and max_features
#numbers below chosen by our mentor DJ
# Number of trees in random forest
#n_estimators = [100, 200, 500]
# Number of features to consider at every split
#max_features = ['auto', 'sqrt','log2']
# Maximum number of levels in tree
#max_depth = [1000, 5000, None]

# Create the random grid
#random_grid = {'n_estimators': n_estimators,
#               'max_features': max_features,
#               'max_depth': max_depth}
#pprint(random_grid)

In [24]:
# Use grid search CV to search for best hyperparameters
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import GridSearchCV   #Perforing grid search

p_test3 = {'max_depth':[1000, 5000, None], 
           'n_estimators':[100, 200, 500],
           'max_features':['auto', 'sqrt', 'log2']}

rf = RandomForestRegressor()
#below is something else we tried but it's irrelevant now
# Random search of parameters, using 3 fold cross validation, 
# search across 100 different combinations, and use all available cores
#rf_random = RandomizedSearchCV(estimator = rf, 
#                               param_distributions = random_grid, 
#                               n_iter = 100, cv = 3, 
#                               verbose=2, 
#                               random_state=42, n_jobs = -1)
rf_grid = GridSearchCV(estimator = rf, 
            param_grid = p_test3, iid=False, cv=3)
# Fit the random search model
rf_grid.fit(X_train_BoW, y_train)
#find the best
rf_grid.cv_results_, rf_grid.best_params_, rf_grid.best_score_

({'mean_fit_time': array([1705.8647875 , 3406.1927042 , 8529.96826458,   68.74360506,
          138.0078551 ,  343.93231853,   42.97686831,   87.0361011 ,
          214.35552661, 1702.99975713, 3389.60591094, 8515.62392855,
           68.9482162 ,  137.93980042,  348.65354029,   45.81338867,
           90.58779724,  226.85547471, 1696.38183467, 3404.24938091,
         8526.92463938,   69.00903527,  137.99634465,  345.36358388,
           45.41530244,   90.02439237,  225.3194476 ]),
  'std_fit_time': array([ 23.13637366,  36.97319351,  57.24847199,   0.80477322,
           1.58767913,   2.7150932 ,   0.42202381,   1.29482833,
           2.42044425,  15.75530349,  35.12218665, 100.99176797,
           0.64473024,   0.89630899,   2.68647244,   0.51444221,
           0.82329451,   2.61813502,  20.06998463,  29.47248095,
          91.46662697,   0.63980853,   0.77247388,   3.2975149 ,
           0.41851366,   1.10504928,   2.14091859]),
  'mean_score_time': array([0.39856021, 0.77466631, 1.

In [26]:
#what were the best hyperparameters?
rf_grid.best_params_

{'max_depth': 1000, 'max_features': 'auto', 'n_estimators': 500}

In [27]:
#import the model we are using with the best hyperparameters

#model
rf_best_tuned = RandomForestRegressor(max_depth=1000,n_estimators=500,max_features='auto')
#train the model
rf_best_tuned.fit(X_train_BoW, y_train)

RandomForestRegressor(bootstrap=True, criterion='mse', max_depth=1000,
                      max_features='auto', max_leaf_nodes=None,
                      min_impurity_decrease=0.0, min_impurity_split=None,
                      min_samples_leaf=1, min_samples_split=2,
                      min_weight_fraction_leaf=0.0, n_estimators=500,
                      n_jobs=None, oob_score=False, random_state=None,
                      verbose=0, warm_start=False)

In [28]:
#Predicting a new result
y_pred = rf_best_tuned.predict(X_test_BoW)

In [29]:
#Evaluating the Algorithm
print('Mean Absolute Error:', metrics.mean_absolute_error(y_test, y_pred))
print('Mean Squared Error:', metrics.mean_squared_error(y_test, y_pred))
print('Root Mean Squared Error:', np.sqrt(metrics.mean_squared_error(y_test, y_pred)))

Mean Absolute Error: 0.22016838176506034
Mean Squared Error: 0.0839900225108869
Root Mean Squared Error: 0.28981032160861164


We can see that the value of root mean squared error is 0.29, which is 7.3% of the mean value of the ratings of all the books, i.e. 3.99. So basically, the best hyperparamters only improved the model by the most neglible amounts. 

In [30]:
import eli5
eli5.show_weights(rf_best_tuned, vec=vectorizer, top=50)

Weight,Feature
0.0089 ± 0.0041,series
0.0083 ± 0.0050,rain
0.0056 ± 0.0032,novel
0.0056 ± 0.0051,amen
0.0050 ± 0.0036,battle
0.0044 ± 0.0028,life
0.0040 ± 0.0025,not
0.0039 ± 0.0022,one
0.0034 ± 0.0052,debut
0.0033 ± 0.0023,new


In [40]:
for name, importance in zip(vectorizer["feature_names"], rf_best_tuned.feature_importances_):
    print(name, "=", importance)


TypeError: 'CountVectorizer' object is not subscriptable

In [36]:
from sklearn.datasets import load_iris
iris = load_iris()

In [39]:
iris

{'data': array([[5.1, 3.5, 1.4, 0.2],
        [4.9, 3. , 1.4, 0.2],
        [4.7, 3.2, 1.3, 0.2],
        [4.6, 3.1, 1.5, 0.2],
        [5. , 3.6, 1.4, 0.2],
        [5.4, 3.9, 1.7, 0.4],
        [4.6, 3.4, 1.4, 0.3],
        [5. , 3.4, 1.5, 0.2],
        [4.4, 2.9, 1.4, 0.2],
        [4.9, 3.1, 1.5, 0.1],
        [5.4, 3.7, 1.5, 0.2],
        [4.8, 3.4, 1.6, 0.2],
        [4.8, 3. , 1.4, 0.1],
        [4.3, 3. , 1.1, 0.1],
        [5.8, 4. , 1.2, 0.2],
        [5.7, 4.4, 1.5, 0.4],
        [5.4, 3.9, 1.3, 0.4],
        [5.1, 3.5, 1.4, 0.3],
        [5.7, 3.8, 1.7, 0.3],
        [5.1, 3.8, 1.5, 0.3],
        [5.4, 3.4, 1.7, 0.2],
        [5.1, 3.7, 1.5, 0.4],
        [4.6, 3.6, 1. , 0.2],
        [5.1, 3.3, 1.7, 0.5],
        [4.8, 3.4, 1.9, 0.2],
        [5. , 3. , 1.6, 0.2],
        [5. , 3.4, 1.6, 0.4],
        [5.2, 3.5, 1.5, 0.2],
        [5.2, 3.4, 1.4, 0.2],
        [4.7, 3.2, 1.6, 0.2],
        [4.8, 3.1, 1.6, 0.2],
        [5.4, 3.4, 1.5, 0.4],
        [5.2, 4.1, 1.5, 0.1],
  

In [ ]:
#gridsearchcv for gradient boosting

p_test3 = {'max_depth':[1000, 5000, None], 
           'n_estimators':[100, 200, 500],
           'max_features':['auto', 'sqrt', 'log2']}
 
tuning = GridSearchCV(estimator =GradientBoostingRegressor(), 
            param_grid = p_test3, iid=False, cv=3)
tuning.fit(X_train_BoW, y_train)
tuning.cv_results_, tuning.best_params_, tuning.best_score_

In [32]:
#import the model we are using with the best hyperparameters

#model
gradient_boosting_regressor_model= GradientBoostingRegressor(max_depth=None,n_estimators=500,max_features='sqrt')
#train the model
gradient_boosting_regressor_model.fit(X_train_BoW, y_train)

GradientBoostingRegressor(alpha=0.9, criterion='friedman_mse', init=None,
                          learning_rate=0.1, loss='ls', max_depth=None,
                          max_features='sqrt', max_leaf_nodes=None,
                          min_impurity_decrease=0.0, min_impurity_split=None,
                          min_samples_leaf=1, min_samples_split=2,
                          min_weight_fraction_leaf=0.0, n_estimators=500,
                          n_iter_no_change=None, presort='auto',
                          random_state=None, subsample=1.0, tol=0.0001,
                          validation_fraction=0.1, verbose=0, warm_start=False)

In [33]:
#Predicting a new result
y_pred = gradient_boosting_regressor_model.predict(X_test_BoW)

In [34]:
#Evaluating the Algorithm
print('Mean Absolute Error:', metrics.mean_absolute_error(y_test, y_pred))
print('Mean Squared Error:', metrics.mean_squared_error(y_test, y_pred))
print('Root Mean Squared Error:', np.sqrt(metrics.mean_squared_error(y_test, y_pred)))

Mean Absolute Error: 0.2221512663234639
Mean Squared Error: 0.0856911595569502
Root Mean Squared Error: 0.29273052378757874


We can see that the value of root mean squared error is 0.295, which is .3% better than our baseline gradient boosting model. So basically, our autotuning the hyperparameters led to the most neglible of improvements.

In [35]:
import eli5
eli5.show_weights(gradient_boosting_regressor_model, vec=vectorizer, top=50)

Weight,Feature
0.0028 ± 0.0041,novel
0.0027 ± 0.0043,rain
0.0021 ± 0.0028,continues
0.0017 ± 0.0031,year
0.0016 ± 0.0045,book
0.0015 ± 0.0058,series
0.0014 ± 0.0026,mystery
0.0013 ± 0.0030,not
0.0011 ± 0.0021,warriors
0.0011 ± 0.0015,world


To sum up, the best model was our autotuned Random Forest model, with hyperparameters {'n_estimators': 500, 'max_features': 'auto', 'max_depth': None}. This led to a root mean squared error of .29. This means that, on average, our model was .29 off from predicting a book's recommendating rating (on a scale of 1 to 5).

In [ ]:
!pip install eli5